In [ ]:
# ----------- init -----------
import osmnx as ox
import geopandas as gpd
import pandas as pd
import folium
import sys
from datetime import date


import folium
from shapely.geometry import LineString, MultiLineString
import plotly.graph_objects as go


# Comparison I : Mirka vs. OSM version type "all", filtered to exclude pedestrian, footpath

In [ ]:
# ----------- load networks -----------

network_mirka = gpd.read_file(r"data\network_comparisons\network_subsets.gpkg", layer = "network_subset_mirka")
network_2504 =  gpd.read_file(r"data\network_comparisons\network_subsets.gpkg", layer = "network_subset_250410")


## Investigate attributes

In [ ]:
display(
    network_2504.groupby(['highway', 'cycleway']).size().reset_index(name='count')
)

## create comparison df

In [ ]:

# ----------- adjustments -----------
network_mirka = network_mirka.rename(columns={
    'FromID': 'u',
    'ToID': 'v',
    'Type': 'highway'
})

network_mirka = network_mirka.to_crs(4326)

network_mirka = network_mirka[['u', 'v', 'highway', 'geometry']]
network_2504 = network_2504[['u', 'v', 'highway', 'geometry']]

network_mirka['u'] = network_mirka['u'].astype('Int64')
network_mirka['v'] = network_mirka['v'].astype('Int64')

# Full outer merge to compare
merged = pd.merge(
    network_mirka,
    network_2504,
    on=['u', 'v', 'highway'],
    how='outer',
    indicator=True
)

# Rename merge indicator values
merged['_merge'] = merged['_merge'].replace({
    'both': 'both',
    'left_only': 'mirka',
    'right_only': '2504'
})

# Split by comparison result
in_both = merged[merged['_merge'] == 'both']
only_in_mirka = merged[merged['_merge'] == 'left_only']
only_in_uli = merged[merged['_merge'] == 'right_only']

# Assign final geometry based on _merge source
merged['geometry'] = merged.apply(
    lambda row: row['geometry_x'] if row['_merge'] in ['both', 'mirka'] else row['geometry_y'],
    axis=1
)

# Set active geometry column
merged = merged.set_geometry('geometry')

# Drop old geometry columns
merged = merged.drop(columns=['geometry_x', 'geometry_y'])





In [ ]:
# ----------- plot comparison -----------

# Center map on average coordinate
center = merged['geometry'].union_all().centroid.coords[0]
m = folium.Map(location=[center[1], center[0]], zoom_start=12)

# Color and width mapping
style_map = {
    'both':     {'color': 'gray',   'weight': 3},
    'mirka':    {'color': 'red',    'weight': 3},
    '2504':   {'color': 'blue',   'weight': 3}
}

for _, row in merged.iterrows():
    geom = row.geometry
    style = style_map[row._merge]
    tooltip = folium.Tooltip(f"{row.highway}\n{row._merge}")

    # Dashed lines
    dash_array = '5,5' if row._merge != 'both' else '5,5'

    if isinstance(geom, LineString):
        coords = [(pt[1], pt[0]) for pt in geom.coords]
        folium.PolyLine(
            locations=coords,
            color=style['color'],
            weight=style['weight'],
            opacity=0.6,
            tooltip=tooltip,
            dash_array=dash_array
        ).add_to(m)

    elif isinstance(geom, MultiLineString):
        for linestring in geom.geoms:
            coords = [(pt[1], pt[0]) for pt in linestring.coords]
            folium.PolyLine(
                locations=coords,
                color=style['color'],
                weight=style['weight'],
                opacity=0.6,
                tooltip=tooltip,
                dash_array=dash_array
            ).add_to(m)
# Display map
display(m)


In [ ]:
# ----------- tabellaric comparison -----------

comparison_table = merged.groupby(['highway', '_merge']).size().unstack(fill_value=0).sort_values(by='highway').reset_index()
comparison_table = comparison_table[~comparison_table['highway'].astype(str).str.contains(r'\[', regex=True)].reset_index(drop=True)
display(comparison_table)

In [ ]:

# Melt for long format
df_long = comparison_table.melt(id_vars='highway', var_name='source', value_name='count')

# Sort highways consistently
df_long['highway'] = df_long['highway'].astype(str)
df_long = df_long.sort_values(by='highway')

# Create grouped bar chart
fig = go.Figure()

for source in df_long['source'].unique():
    subset = df_long[df_long['source'] == source]
    fig.add_trace(go.Bar(
        x=subset['highway'],
        y=subset['count'],
        name=source
    ))

fig.update_layout(
    barmode='group',
    title='Link Count by Highway Type and Source',
    xaxis_title='Highway Type',
    yaxis_title='Count',
    xaxis_tickangle=-45,
    height=500,
    plot_bgcolor='white',
    paper_bgcolor='white',
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        griddash='dash'  # Dashed y-axis lines
    )
)

fig.show()

# Comparison II : Mirka vs. OSM version type "all", filtered to INCLUDE pedestrian, footpath

In [ ]:
# ----------- load networks -----------

network_mirka = gpd.read_file(r"data\network_comparisons\network_subsets.gpkg", layer = "network_subset_mirka")
network_2504 =  gpd.read_file(r"data\network_comparisons\network_subsets.gpkg", layer = "network_subset_250409")


In [ ]:
# ----------- adjustments -----------
network_mirka = network_mirka.rename(columns={
    'FromID': 'u',
    'ToID': 'v',
    'Type': 'highway'
})

network_mirka = network_mirka.to_crs(4326)

network_mirka = network_mirka[['u', 'v', 'highway', 'geometry']]
network_2504 = network_2504[['u', 'v', 'highway', 'geometry']]

network_mirka['u'] = network_mirka['u'].astype('Int64')
network_mirka['v'] = network_mirka['v'].astype('Int64')


In [ ]:

# Full outer merge to compare
merged = pd.merge(
    network_mirka,
    network_2504,
    on=['u', 'v', 'highway'],
    how='outer',
    indicator=True
)

# Rename merge indicator values
merged['_merge'] = merged['_merge'].replace({
    'both': 'both',
    'left_only': 'mirka',
    'right_only': '2504'
})

# Split by comparison result
in_both = merged[merged['_merge'] == 'both']
only_in_mirka = merged[merged['_merge'] == 'left_only']
only_in_uli = merged[merged['_merge'] == 'right_only']

# Assign final geometry based on _merge source
merged['geometry'] = merged.apply(
    lambda row: row['geometry_x'] if row['_merge'] in ['both', 'mirka'] else row['geometry_y'],
    axis=1
)

# Set active geometry column
merged = merged.set_geometry('geometry')

# Drop old geometry columns
merged = merged.drop(columns=['geometry_x', 'geometry_y'])





In [ ]:
# ----------- plot comparison -----------

# Center map on average coordinate
center = merged['geometry'].union_all().centroid.coords[0]
m = folium.Map(location=[center[1], center[0]], zoom_start=12)

# Color and width mapping
style_map = {
    'both':     {'color': 'gray',   'weight': 3},
    'mirka':    {'color': 'red',    'weight': 3},
    '2504':   {'color': 'blue',   'weight': 3}
}

for _, row in merged.iterrows():
    geom = row.geometry
    style = style_map[row._merge]
    tooltip = folium.Tooltip(f"{row.highway}\n{row._merge}")

    # Dashed lines
    dash_array = '5,5' if row._merge != 'both' else '5,5'

    if isinstance(geom, LineString):
        coords = [(pt[1], pt[0]) for pt in geom.coords]
        folium.PolyLine(
            locations=coords,
            color=style['color'],
            weight=style['weight'],
            opacity=0.6,
            tooltip=tooltip,
            dash_array=dash_array
        ).add_to(m)

    elif isinstance(geom, MultiLineString):
        for linestring in geom.geoms:
            coords = [(pt[1], pt[0]) for pt in linestring.coords]
            folium.PolyLine(
                locations=coords,
                color=style['color'],
                weight=style['weight'],
                opacity=0.6,
                tooltip=tooltip,
                dash_array=dash_array
            ).add_to(m)
# Display map
display(m)


In [ ]:
# ----------- tabellaric comparison -----------

comparison_table = merged.groupby(['highway', '_merge']).size().unstack(fill_value=0).sort_values(by='highway').reset_index()
comparison_table = comparison_table[~comparison_table['highway'].astype(str).str.contains(r'\[', regex=True)].reset_index(drop=True)
display(comparison_table)

In [ ]:

# Melt for long format
df_long = comparison_table.melt(id_vars='highway', var_name='source', value_name='count')

# Sort highways consistently
df_long['highway'] = df_long['highway'].astype(str)
df_long = df_long.sort_values(by='highway')

# Create grouped bar chart
fig = go.Figure()

for source in df_long['source'].unique():
    subset = df_long[df_long['source'] == source]
    fig.add_trace(go.Bar(
        x=subset['highway'],
        y=subset['count'],
        name=source
    ))

fig.update_layout(
    barmode='group',
    title='Link Count by Highway Type and Source',
    xaxis_title='Highway Type',
    yaxis_title='Count',
    xaxis_tickangle=-45,
    height=500,
    plot_bgcolor='white',
    paper_bgcolor='white',
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        griddash='dash'  # Dashed y-axis lines
    )
)

fig.show()